<a href="https://colab.research.google.com/github/andersonxpm/mestrado-ime/blob/master/cd_trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Trabalho: Ciência de Dados / Anderson Xavier de Paiva Mello




1. Instalação dos pacotes

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial

from sklearn.metrics import accuracy_score, average_precision_score, classification_report, confusion_matrix, make_scorer, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM

2. Upload do dataset

In [10]:
## PROC ##

bi = pd.read_csv('base_nt_poc.csv', encoding='latin-1')
print("Dados carregados com sucesso!")

Dados carregados com sucesso!


In [11]:
data1 = bi.copy()
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88045 entries, 0 to 88044
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   data_solicitacao                  88045 non-null  object 
 1   id_prestador_executor             88045 non-null  object 
 2   classificacao_prestador_executor  88045 non-null  object 
 3   cpf_tutor                         88045 non-null  object 
 4   cidade_tutor                      88045 non-null  object 
 5   estado_tutor                      88045 non-null  object 
 6   id_pet_tutor                      88045 non-null  object 
 7   idade_pet_anos                    88045 non-null  float64
 8   especie                           88045 non-null  object 
 9   raca_pet                          88045 non-null  object 
 10  genero_pet                        88040 non-null  object 
 11  id_plano                          88045 non-null  object 
 12  valo

In [12]:
# Garantir que a coluna está no formato datetime
data1['data_solicitacao'] = pd.to_datetime(data1['data_solicitacao'])

# Datas mínima e máxima
data_min = data1['data_solicitacao'].min()
data_max = data1['data_solicitacao'].max()

print("📅 Data mínima:", data_min)
print("📅 Data máxima:", data_max)

📅 Data mínima: 2024-01-01 00:00:00
📅 Data máxima: 2024-01-31 00:00:00


3.   Limpeza dos dados



In [13]:
## PROC ##

# Gravando bi = data
data = bi.copy()

In [6]:
## PROC ##

# Tratar valores nulos - aqui, removemos as linhas com valores nulos em 'genero_pet' e valores negativos em 'idade_pet_anos' e 'valor'
data = data.dropna(subset=['genero_pet'])
data = data[(data['idade_pet_anos'] >= 0)]
data = data[(data['valor_mensal_plano'] >= 0)]
data = data[(data['valor'] >= 0)]

In [7]:
## PROC ##

# Remover colunas irrelevantes
## As variáveis numéricas não possuem alta correlação de acordo com o coeficiente de Pearson, de -1 a +1)
cols_to_remove = ['data_solicitacao', 'cpf_tutor', 'id_pet_tutor', 'id_procedimento']
data = data.drop(columns=cols_to_remove)

4. Análise exploratória

In [ ]:
# Análise: tipo das variáveis
data.dtypes

In [ ]:
# Análise: explorar dataset
data.info()

In [ ]:
# Análise: descrição do dataset
data.describe(include='all')

In [ ]:
# Verificando valores nulos
print("Valores nulos por coluna:\n", data.isnull().sum())

In [ ]:
# Verificando valores negativos
negativos = data.select_dtypes(include=[np.number]).lt(0).sum()
print("Valores negativos por coluna:\n", negativos[negativos > 0])

In [ ]:
# Proporção de glosas
data['fl_glosa_definitiva'].value_counts(normalize=True)

In [ ]:
# Proporção de glosas - distribuição da variável alvo
sns.countplot(data=data, x='fl_glosa_definitiva')
plt.title('Distribuição da variável alvo (glosa definitiva)')
plt.show()

In [ ]:
# Cardinalidade das variáveis
data.nunique().sort_values(ascending=False)

In [ ]:
# Correlação entre variáveis numéricas
num_cols = ['idade_pet_anos', 'valor_mensal_plano', 'valor']

sns.heatmap(data[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlação entre variáveis numéricas')
plt.show()

In [ ]:
# Análise univariada de variáveis contínuas
num_cols = ['idade_pet_anos', 'valor_mensal_plano', 'valor']

for col in num_cols:
    sns.histplot(data[col], kde=True)
    plt.title(f'Distribuição da variável {col}')
    plt.show()

for col in num_cols:
    sns.boxplot(data=data2, x='fl_glosa_definitiva', y=col)
    plt.title(f'{col} por glosa')
    plt.show()

In [ ]:
# Análise univariada de variáveis contínuas - estatísticas básicas
num_cols = data.select_dtypes(include=['number'])

# Média
media = num_cols.mean()

# Desvio padrão
desvio = num_cols.std()

# Moda (pode haver mais de uma, por isso [0])
moda = num_cols.mode().iloc[0]

resumo = pd.DataFrame({
    'média': media,
    'moda': moda,
    'desvio_padrão': desvio
})

print(resumo)

In [ ]:
# Análise univariada de variáveis categóricas
cat_cols = ['classificacao_prestador_executor', 'cidade_tutor', 'estado_tutor',
            'especie', 'raca_pet', 'genero_pet', 'ds_grupo_procedimento']

for col in cat_cols:
    plt.figure(figsize=(12, 5))
    sns.countplot(data=data, y=col, order=bi[col].value_counts().index[:10])
    plt.title(f'{col} - Top 10 categorias')
    plt.show()

In [ ]:
# Converter a coluna de datas para datetime (se ainda não estiver)
data1 = bi.copy()
##data1['data_solicitacao'] = pd.to_datetime(data1['data_solicitacao'])

# Agrupar por data e contar o número de procedimentos por dia
volume_diario = data1.groupby('data_solicitacao').size()

# Plotar
plt.figure(figsize=(9, 4))
volume_diario.plot(kind='bar')
plt.title('Volume Diário de Procedimentos')
plt.xlabel('Data da Solicitação')
plt.ylabel('Número de Procedimentos')
plt.tight_layout()
plt.show()

In [ ]:
# Análise bivariada categórica × glosa
prop_df = pd.crosstab(data1['data_solicitacao'], data1['fl_glosa_definitiva'], normalize='index')
prop_df.plot(kind='bar', stacked=True, figsize=(10.5, 4))
plt.title('Proporção de glosa por data_solicitacao')
plt.ylabel('Proporção')
plt.show()

In [ ]:
# Análise bivariada categórica × glosa
cat_cols = ['classificacao_prestador_executor', 'cidade_tutor', 'estado_tutor',
            'especie', 'raca_pet', 'genero_pet', 'ds_grupo_procedimento']

for col in cat_cols:
    prop_df = pd.crosstab(data[col], data['fl_glosa_definitiva'], normalize='index')
    prop_df.plot(kind='bar', stacked=True, figsize=(10, 5))
    plt.title(f'Proporção de glosa por {col}')
    plt.ylabel('Proporção')
    plt.show()

In [ ]:
# Balanceamento da base (ver repetição de prestadores, procedimentos etc.)
prestador_glosa = data.groupby('id_prestador_executor')['fl_glosa_definitiva'].mean()
prestador_glosa.hist(bins=30)
plt.title('Taxa média de glosa por prestador')
plt.xlabel('Taxa de glosa')
plt.ylabel('Frequência')
plt.show()

5. Deteccao de outliers

In [ ]:
## PROC ##

# Deteccao de outlier
coluna = 'idade_pet_anos'  # Substitua pelo nome da coluna a analisar

Q1 = data[coluna].quantile(0.25)
Q3 = data[coluna].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print(f"Limites para '{coluna}': {limite_inferior:.2f} a {limite_superior:.2f}")

data[f'{coluna}_outlier'] = (data[coluna] <= limite_inferior) | (data[coluna] >= limite_superior)

##data[['idade_pet_anos', 'idade_pet_anos_outlier']].head(10)
##data.loc[data[f'{coluna}_outlier'] == True, ['idade_pet_anos', 'idade_pet_anos_outlier']]

In [ ]:
## PROC ##

# Deteccao de outlier
coluna = 'valor_mensal_plano'  # Substitua pelo nome da coluna a analisar

Q1 = data[coluna].quantile(0.25)
Q3 = data[coluna].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print(f"Limites para '{coluna}': {limite_inferior:.2f} a {limite_superior:.2f}")

data[f'{coluna}_outlier'] = (data[coluna] <= limite_inferior) | (data[coluna] >= limite_superior)

##data[['valor_mensal_plano', 'valor_mensal_plano_outlier']].head(10)
##data.loc[data[f'{coluna}_outlier'] == True, ['valor_mensal_plano', 'valor_mensal_plano_outlier']]

In [ ]:
## PROC ##

# Deteccao de outlier
coluna = 'valor'  # Substitua pelo nome da coluna a analisar

Q1 = data[coluna].quantile(0.25)
Q3 = data[coluna].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print(f"Limites para '{coluna}': {limite_inferior:.2f} a {limite_superior:.2f}")

data[f'{coluna}_outlier'] = (data[coluna] <= limite_inferior) | (data[coluna] >= limite_superior)

##data[['valor', 'valor_outlier']].head(10)
##data.loc[data[f'{coluna}_outlier'] == True, ['valor', 'valor_outlier']]

In [29]:
## PROC ##

# Removendo outliers
data2 = data.copy()

data2 = data2[data2['idade_pet_anos_outlier'] == False]
data2 = data2[data2['valor_mensal_plano_outlier'] == False]
data2 = data2[data2['valor_outlier'] == False]

In [30]:
## PROC ##

# Removendo a coluna auxiliar dos outliers
data2 = data2.drop(columns=['idade_pet_anos_outlier'])
data2 = data2.drop(columns=['valor_mensal_plano_outlier'])
data2 = data2.drop(columns=['valor_outlier'])

In [ ]:
# Contagem da variavel de controle
data2['fl_glosa_definitiva'].value_counts()

Preparação das bases para os modelos
---

In [ ]:
data2.info()

In [34]:
# Separando X (features) e y (rótulo real de glosa)
data3 = data2.copy()

col_order = ['id_plano',
             'id_prestador_executor',
             'classificacao_prestador_executor',
             'ds_grupo_procedimento',
             'cidade_tutor',
             'estado_tutor',
             'especie',
             'raca_pet',
             'genero_pet',
             'idade_pet_anos',
             'valor',
             'valor_mensal_plano',
             'fl_glosa_definitiva']

data3 = data3[col_order]

X = data3.drop(columns=['fl_glosa_definitiva', 'id_plano', 'id_prestador_executor']) # Drop ID columns
y_true = data3['fl_glosa_definitiva']

# Split com estratificação para manter a proporção de glosas
X_train, X_test, y_train, y_test = train_test_split(
    X, y_true, stratify=y_true, test_size=0.3, random_state=42)

# Identifica as variáveis/colunas categóricas
categorical_cols = X_train.select_dtypes(include='object').columns

# Aplica o one-hot encoding nas bases de treino e teste
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Reindex X_test para dar match com X_train depois do one-hot encoding
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Regressão Logística - Modelo
---

In [35]:
# Copiando as variáveis
X_rl = X.copy()
y_true_rl = y_true.copy()

X_train_rl = X_train.copy()
X_test_rl = X_test.copy()
y_train_rl = y_train.copy()
y_test_rl = y_test.copy()

X_train_scaled_rl = X_train_scaled.copy()
X_test_scaled_rl = X_test_scaled.copy()

# Identificando as variáveis categóricas
categorical_cols_rl = X_rl.select_dtypes(include='object').columns

# Aplicando one-hot encoding nas variáveis categóricas
X_rl = pd.get_dummies(X_rl, columns=categorical_cols_rl, drop_first=True)

# Pipeline SEM balanceamento
pipeline_sem_balanceamento = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# Pipeline COM balanceamento MOTE
pipeline_com_smote = ImbPipeline(steps=[
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# Pipeline COM balanceamento class_weight='balanced'
pipeline_com_class_weight = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42))
])

In [36]:
# Treinamento dos pipelines
pipeline_sem_balanceamento.fit(X_train_rl, y_train_rl)
pipeline_com_smote.fit(X_train_rl, y_train_rl)
pipeline_com_class_weight.fit(X_train_rl, y_train_rl)

# Predições
y_pred_sem_rl = pipeline_sem_balanceamento.predict(X_test_rl)
y_pred_smote_rl = pipeline_com_smote.predict(X_test_rl)
y_pred_weight_rl = pipeline_com_class_weight.predict(X_test_rl)

y_proba_sem_rl = pipeline_sem_balanceamento.predict_proba(X_test_rl)[:, 1]
y_proba_smote_rl = pipeline_com_smote.predict_proba(X_test_rl)[:, 1]
y_proba_weight_rl = pipeline_com_class_weight.predict_proba(X_test_rl)[:, 1]

In [ ]:
# Métricas - RL sem balanceamento
y_true_binary = y_test_rl
y_pred_binary = y_pred_sem_rl
y_pred_proba = y_proba_sem_rl

print("\n🔍 Classification Report:")
print(classification_report(y_true_binary, y_pred_binary, digits=4))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_true_binary, y_pred_binary))

roc_auc = roc_auc_score(y_true_binary, y_pred_proba)
pr_auc = average_precision_score(y_true_binary, y_pred_proba)
print(f"\n📈 ROC AUC: {roc_auc:.4f}")
print(f"📉 PR AUC (Average Precision): {pr_auc:.4f}")

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_true_binary, y_pred_proba)
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Métricas - RL com balanceamento SMOTE
y_true_binary = y_test_rl
y_pred_binary = y_pred_smote_rl
y_pred_proba = y_proba_smote_rl


print("\n🔍 Classification Report:")
print(classification_report(y_true_binary, y_pred_binary, digits=4))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_true_binary, y_pred_binary))

roc_auc = roc_auc_score(y_true_binary, y_pred_proba)
pr_auc = average_precision_score(y_true_binary, y_pred_proba)
print(f"\n📈 ROC AUC: {roc_auc:.4f}")
print(f"📉 PR AUC (Average Precision): {pr_auc:.4f}")

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_true_binary, y_pred_proba)
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Métricas - RL com balanceamento class_weight='balanced'
y_true_binary = y_test_rl
y_pred_binary = y_pred_weight_rl
y_pred_proba = y_proba_weight_rl


print("\n🔍 Classification Report:")
print(classification_report(y_true_binary, y_pred_binary, digits=4))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_true_binary, y_pred_binary))

roc_auc = roc_auc_score(y_true_binary, y_pred_proba)
pr_auc = average_precision_score(y_true_binary, y_pred_proba)
print(f"\n📈 ROC AUC: {roc_auc:.4f}")
print(f"📉 PR AUC (Average Precision): {pr_auc:.4f}")

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_true_binary, y_pred_proba)
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()

Isolation Forest - Modelo
---

In [40]:
# Copiando as variáveis
X_if = X.copy()
y_true_if = y_true.copy()

X_train_if = X_train.copy()
X_test_if = X_test.copy()
y_train_if = y_train.copy()
y_test_if = y_test.copy()

X_train_scaled_if = X_train_scaled.copy()
X_test_scaled_if = X_test_scaled.copy()

# Identificando as variáveis categóricas
categorical_cols_if = X_if.select_dtypes(include='object').columns

# Aplicando one-hot encoding nas variáveis categóricas
X_if = pd.get_dummies(X_if, columns=categorical_cols_if, drop_first=True)

# Treinando o modelo não supervisionado
iso = IsolationForest(contamination=0.015, random_state=42)  # Ajustando a contaminação conforme a taxa de glosa real
iso.fit(X_train_scaled_if) # Ajustando apenas os dados de treinamento

# Predição no conjunto de teste
y_pred_test_if = iso.predict(X_test_scaled_if)

# Isolation Forest retorna:
# -1 para anomalia
#  1 para normal
# Vamos transformar para 1 = glosa (anomalia), 0 = normal, para bater com y_true
y_pred_bin_if = np.where(y_pred_test_if == -1, 1, 0)

In [ ]:
# Métricas
print("\n🔍 Classification Report:")
print(classification_report(y_test_if, y_pred_bin_if, digits=4))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test_if, y_pred_bin_if))

# Isolation Forest does not output probabilities, so we cannot calculate ROC AUC and PR AUC in the traditional way
# However, we can use the decision_function output as a score
y_scores_if = iso.decision_function(X_test_scaled_if)

# In Isolation Forest, lower scores indicate anomalies (glosa)
# We need to invert the scores for ROC AUC and PR AUC where higher scores indicate the positive class (glosa=1)
y_scores_inverted_if = -y_scores_if

roc_auc = roc_auc_score(y_test_if, y_scores_inverted_if)
pr_auc = average_precision_score(y_test_if, y_scores_inverted_if)
print(f"\n📈 ROC AUC: {roc_auc:.4f}")
print(f"📉 PR AUC (Average Precision): {pr_auc:.4f}")

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_scores_inverted_if)
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()

Randon Forest - Modelo
---

In [42]:
# Copiando as variáveis
X_rf = X.copy()
y_true_rf = y_true.copy()

X_train_rf = X_train.copy()
X_test_rf = X_test.copy()
y_train_rf = y_train.copy()
y_test_rf = y_test.copy()

X_train_scaled_rf = X_train_scaled.copy()
X_test_scaled_rf = X_test_scaled.copy()

# SMOTE

# Identificando as variáveis categóricas
categorical_cols_rf = X_rf.select_dtypes(include='object').columns

# Aplicando one-hot encoding nas variáveis categóricas
X_rf = pd.get_dummies(X_rf, columns=categorical_cols_rf, drop_first=True)

# --- #

# Pipeline: Randon Forest sem balanceamento
rf_sem = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1)

# Pipeline: RandomForest com balanceamento SMOTE

# Normalizando os dados para o SMOTE
X_train_rf_res, y_train_rf_res = SMOTE().fit_resample(X_train_rf, y_train_rf)

scaler_rf = StandardScaler()
X_train_scaled_rf_smote = scaler_rf.fit_transform(X_train_rf_res)
X_test_scaled_rf_smote = scaler_rf.transform(X_test_rf)

rf_com_smote = ImbPipeline(steps=[
    ('smote', SMOTE(sampling_strategy='auto', random_state=42)),
    ('clf', RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])

# Pipeline: RandomForest com balaceamento class_weight='balanced'
rf_com_weight = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

In [43]:
# Treinamento e predição RF sem balanceamento
rf_sem.fit(X_train_scaled_rf, y_train_rf)
y_pred_sem_rf = rf_sem.predict(X_test_scaled_rf)
y_proba_sem_rf = rf_sem.predict_proba(X_test_scaled_rf)[:, 1]

In [ ]:
# Validação do SMOTE
print("Antes do SMOTE:", np.bincount(y_train_rf))
print("Depois do SMOTE:", np.bincount(y_train_rf_res))

In [45]:
# Treinamento e predição RF comm balanceamento SMOTE
rf_com_smote.fit(X_train_scaled_rf_smote, y_train_rf_res)
y_pred_smote_rf = rf_com_smote.predict(X_test_scaled_rf_smote)
y_proba_smote_rf = rf_com_smote.predict_proba(X_test_scaled_rf_smote)[:, 1]

In [46]:
# Treinamento e predição RF comm balanceamento Weight
rf_com_weight.fit(X_train_scaled_rf, y_train_rf)
y_pred_weight_rf = rf_com_weight.predict(X_test_scaled_rf)
y_proba_weight_rf = rf_com_weight.predict_proba(X_test_scaled_rf)[:, 1]

In [ ]:
# Métricas - RF sem balaceamento
y_test = y_test_rf
y_pred = y_pred_sem_rf
y_proba = y_proba_sem_rf

print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)
print(f"\n📈 ROC AUC: {roc_auc:.4f}")
print(f"📉 PR AUC (Average Precision): {pr_auc:.4f}")

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Métricas - RF com balaceamento SMOTE
y_test = y_test_rf
y_pred = y_pred_smote_rf
y_proba = y_proba_smote_rf

print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)
print(f"\n📈 ROC AUC: {roc_auc:.4f}")
print(f"📉 PR AUC (Average Precision): {pr_auc:.4f}")

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Métricas - RF com balaceamento class_weight='balanced'
y_test = y_test_rf
y_pred = y_pred_weight_rf
y_proba = y_proba_weight_rf

print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)
print(f"\n📈 ROC AUC: {roc_auc:.4f}")
print(f"📉 PR AUC (Average Precision): {pr_auc:.4f}")

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(8, 5))
plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()